In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


In [4]:
# Select DL model
# If you change the model, re-run all the cells below
# Other applicable models: ner_dl, ner_dl_bert
MODEL_NAME = "onto_100"

In [5]:
# Text taken from user's upload
text_list = [
    """14
 6 Community  paediatrics  example

Community  Paediatrics  Clinic,  Adobe  Health  Centre,  Donaldstown,  DO1  4XP (01234)  567890
Adam  Rimmer,  Community  Paediatrician  cpc@nhs.uk

Outpatient  letter  to General  practitioner
Patient demographics  Attendance details
Patient name  Miss  Mary Jones  Date of
appointment/contact  01/05/2017
Date of birth  01/0 2/2013  Contact type  Follow -up
Gender  Female  Consultation method  Face -to-face
NHS number . 124352319 Seen by  Dr. Adam Rimmer, Consultant
paediatrician
Hospital ID  TL5694 5  (01234) 56 4563
Patient  address  31 Acacia Road , BM9
6PL Care professional s
present  Jenny White, Occupational therapist
Sarah Hall, H ealth care assistant
  Person accompanying
patient  Sally Jones, mother
  Outcome  of outpatient
attendance  Appointment will be made at a later
date
Relevant contacts  Sally and Ian Jones
(parents)  GP Practice
Patient e mail
address  sally @jones .net GP practice identifier  A111111
Patient t elephone
number  077 1234  7777 GP name  Dr  C. O’Reilly
Educational
establishment  Greenacre School,
Donaldstown  DO5  6AA GP details  Canvas Health Centre, 27 Acacia Road,
BM9 6PM (01234) 956412

Diagnoses  Problems and issues
1. Grand mal epilepsy ,
2. Gastro -oesophageal reflux ,
3. Spastic quadriplegia secondary to birth asphyxia ,
4. Cortical visual impairment ,
5. Bilateral convergent squint ,
6. General learning difficulties . 1. Increased tonic -clonic convulsions ,
2. Problems with transport to school ,
3. Increasingly tight righ t hip .

Dear  Dr. O’Reilly,

I had the pleasure  of seeing  Mary  and her mother  Sally  in my outpatient  clinic  today.

Clinical summary
Mary attended today for a scheduled review . Mary ’s epilepsy is not well controlled at present . She is  having on
average four tonic -clonic  seizures a day. Her mother has had to give her rectal diazepam on two occasions but she
has not needed to go to hospital. She had a PEG inserted in April 2017 and her reflux has reduced considerably
since then.  She has increasing spasticity of right hip .

Allergies and adverse reactions  No known allergies or adverse reactions.

Social context
Educational history  Mary started at Greenacre School in September. She enjoys it and the teachers are
pleased with her progress.  Her mother has been taking  her to school by car but this is

 15
 becoming an increasing problem as due to recent changes  at work she now has to start
work at 8:30. Unfortunately Mary is not eligible for free school transport until she is five
years old. The home -school liaison teacher is trying to come to an agreement with the
local authority to enable Mary to use school trans port.  The teacher for visual impairment
has seen her in school and recommended that she use large print books and a magnifying
glass.

Review of systems
Neurodevelopmental  assessment
Mary can now sit unsupported for about 30 seconds. When lying prone she can draw her knees up underneath her
but does not make any attempts to move. In clinic she was able to complete the circle and square form board but
cannot do them reversed. She can say 10 words with meaning and her mother feels she can understan d far more.
She is able to finger feed and will drink from a cup if it is held for her. She is becoming more sociable and has a
lovely smile.

Examination findings
Musculosketal system & nervous
system  Mary's ankles both dorsiflex  to 90°.  Her hips are very tight; the right hip only
abducts to 30° and the left hip to 45°. The right hip has deteriorated .
Dental  No evidence of dental caries.

Plan and requested action s

Actions for patient or their carer
Mrs. Jones to contact epilepsy nurse on 01226 730000 if she has any concerns regarding convulsions  or
the medication change  documented below .


Action s for healthcare professionals                                                               Action by:
Request to orthopaedics for early appointment for advice on
deteriorating right hip
 Dr. Adam Rimmer, 01/05/17
Letter of support to  the local authority  education department
regarding school transport
 Dr. Adam Rimmer, 01/05/17
Review medication with Sally’s mother by telephone in two weeks . Mr Phil Brown, epilepsy nurse

Medication and medical devices
(only changes to medications and medical devices as a result of the outpatient encounter are included)
Medication name  Lamotrigine
Form Dispersible tablets
Route  Via gastrostomy
Dose  10 mg
Dose direction  Twice a day
Status  Amended
Reason for medication
change  Increasing tonic -clonic  convulsions
Date of latest change  01/05/17
Medication change  Increase to 10mg twice daily
Comment /recommendation  14-day prescription provided, please p rovide a new prescription on parent’s
request .
Total dose daily quantity  20mg



 16
 Yours  sincerely


Person  completing  record : Dr. Adam  Rimmer,  Consultant  community  paediatrician;  Date:  01/05/17 : 16:42

Distribution  list: Sally  & Ian Jones  (patient’s  parents);
Dr.  C. O’Reilly  (GP)
Dr. Charlotte  Worth,  Consultant  orthopaedic  physician,  St. Crispin’s  Hospital;
Mr. Philip  Brown,  epilepsy  nurse,  St. Crispin’s  Hospital
    """]

In [6]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# ner_dl and onto_100 model are trained with glove_100d, so the embeddings in
# the pipeline should match
if (MODEL_NAME == "ner_dl") or (MODEL_NAME == "onto_100"):
    embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

# Bert model uses Bert embeddings
elif MODEL_NAME == "ner_dl_bert":
    embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')

ner_model = NerDLModel.pretrained(MODEL_NAME, 'en') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        documentAssembler,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [7]:
# Run pipeline
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

In [8]:
# Visualisation
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)